# Introduction to Recommendation systems

In this modern world we are overloaded with data and this data provides us the useful information. But it's not possible for the user to extract the information which interest them from these data. In order to help the user to find out information about the product , recommedation systems where developed. 

Recommeder system creates a similarity between the user and items and exploits the similarity between user/item to make recommendations.


What recommeder system can solve ?

1. It can help the user to find the right product.
2. It can increase the user engagement. For example, there's 40% more click on the google news due to recommendation.
3. It helps the item providers to deliver the items to the right user.In Amazon , 35 % products get sold due to recommendation.
4. It helps to make the contents more personalized.In Netflix most of the rented movies are from recommendations.


# Types of recommendations

There are mainly 6 types of the recommendations systems :-

1. Popularity based systems :- It works by recommeding items viewed and purchased by most people and are rated high.It is not a personalized recommendation.
2. Classification model based:- It works by understanding the features of the user and applying the classification algorithm to decide whether the user is     interested or not in the prodcut.
3. Content based recommedations:- It is based on the information on the contents of the item rather than on the user opinions.The main idea is if the user likes an item then he or she will like the "other" similar item.
4. Collaberative Filtering:- It is based on assumption that people like things similar to other things they like, and things that are liked by other people with similar taste. it is mainly of two types:
 a) User-User 
 b) Item -Item
 
5. Hybrid Approaches:- This system approach is to combine collaborative filtering, content-based filtering, and other approaches . 
6. Association rule mining :- Association rules capture the relationships between items based on their patterns of co-occurrence across transactions.



# Attribute Information:

● customer_id : Every user identified with a unique id 

● product_id : Every product identified with a unique id 

● review_score : Rating of the corresponding product by the corresponding user 




# Import Libraries 

In [ ]:
import dask.dataframe as dd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Load the Dataset and Add headers

In [ ]:
review_data = pd.read_csv("../input/brazilian-ecommerce/olist_order_reviews_dataset.csv")
orders_data = pd.read_csv("../input/brazilian-ecommerce/olist_orders_dataset.csv")


Merging reviews and orders 

In [ ]:
data1 = review_data.merge(orders_data, on= 'order_id')
data1.head()

Since, we don't need these two data frames any longer so i deleted them.

In [ ]:
del review_data
del orders_data
gc.collect()

In [ ]:
order_items_data = pd.read_csv("../input/brazilian-ecommerce/olist_order_items_dataset.csv")
order_items_data.head()


Merging data1 and order_items

In [ ]:
data2 = data1.merge(order_items_data,on = 'order_id')
data2

In [ ]:
del order_items_data
gc.collect()

from above dataset we just need customer_id, product_id and review_score(Ratings) as they are necessary for user-product interaction matrix. 

In [ ]:
electronics_data = data2[['customer_id','product_id','review_score']]
electronics_data

In [ ]:
del data1
del data2
gc.collect()

In [ ]:
# Display the data

electronics_data.head()


In [ ]:
#Shape of the data
electronics_data.shape

In [ ]:
#Check the datatypes
electronics_data.dtypes

In [ ]:
electronics_data.info()


In [ ]:

gc.collect()

In [ ]:
#Five point summary 

electronics_data.describe()['review_score'].T


## Handling Missing values


In [ ]:
#Check for missing values
print('Number of missing values across columns: \n',electronics_data.isnull().sum())



## Ratings

In [ ]:
# Check the distribution of the rating
with sns.axes_style('white'):
    g = sns.factorplot("review_score", data=electronics_data, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")

## Unique Users and products


In [ ]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",electronics_data.shape[0])
print("Total No of Users   :", len(np.unique(electronics_data.customer_id)))
print("Total No of products  :", len(np.unique(electronics_data.product_id)))

# Analyzing the rating

In [ ]:
#Analysis of rating given by the user 

no_of_ratings_given_by_users = electronics_data.groupby(by='customer_id')['review_score'].count().sort_values(ascending=False)

no_of_ratings_given_by_users

It means user(be1c4e52bb71e0c54b11a26b8e8d59f2) have rated 22 times on amazon's website for different products

In [ ]:
no_of_ratings_given_by_users.describe()

In [ ]:
#Analysis of ratings for every product

no_of_rated_products_per_user = electronics_data.groupby(by='product_id')['review_score'].count().sort_values(ascending=False)

no_of_rated_products_per_user

It means Product(aca2eb7d00ea1a7b8ebd4e68314663af) have been rated 527 times.

In [ ]:
product_avg_rate = electronics_data.groupby(by='product_id')['review_score'].mean().sort_values(ascending=False)

In [ ]:
product_avg_rate.describe()

In [ ]:
new_df = electronics_data

In [ ]:
del electronics_data
gc.collect()

# Collaberative filtering (Item-Item recommedation)

Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. We are going to use collaborative filtering (CF) approach.
CF is based on the idea that the best recommendations come from people who have similar tastes. In other words, it uses historical item ratings of like-minded people to predict how someone would rate an item.Collaborative filtering has two sub-categories that are generally called memory based and model-based approaches.



Surprise is a python library for Recommendation system

In [ ]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import SVD
import os
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [ ]:
#Reading the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_df,reader)

In [ ]:
# single value decomposition
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#Splitting the dataset
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [ ]:
del data
gc.collect()

Different variants of KNN algorithm

In [ ]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
models = []
models.append(('KNNWithMeans',KNNWithMeans(k=5, sim_options={'name': 'cosine', 'user_based': False})))
models.append(('KNNBasic', KNNBasic(k=5, sim_options={'name': 'cosine', 'user_based': False})))
# models.append(('KNNWithZScore', KNNWithZScore(k=5, sim_options={'name': 'cosine', 'user_based': False})))
# models.append(('KNNBaseline', KNNBaseline(k=5, sim_options={'name': 'cosine', 'user_based': False})))

In [ ]:
for name, model in models:
    model.fit(trainset)
    test_pred = model.test(testset)
    print("RMSE of ",name)
    accuracy.rmse(test_pred, verbose=True)

Results of other two algorithms are as follows :     
* RMSE of KNNWithZScore        <br />RMSE : 1.26(item based, all instances)
* RMSE of KNNBaseline          <br />RMSE : 1.24(item based, all instances)

In [ ]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
models = []
models.append(('KNNWithMeans',KNNWithMeans(k=5, sim_options={'name': 'cosine', 'user_based': True})))
models.append(('KNNBasic', KNNBasic(k=5, sim_options={'name': 'cosine', 'user_based': True})))
models.append(('KNNWithZScore', KNNWithZScore(k=5, sim_options={'name': 'cosine', 'user_based': True})))
models.append(('KNNBaseline', KNNBaseline(k=5, sim_options={'name': 'cosine', 'user_based': True})))
for name, model in models:
    model.fit(trainset)
    test_pred = model.test(testset)
    print("RMSE of ",name)
    accuracy.rmse(test_pred, verbose=True)

Result of user based collaborative filtering
* RMSE of KNNMeans        <br />RMSE : 1.31(User based, with 55,000 instances)
* RMSE of KNNBasic        <br />RMSE : 1.36(User based, with 55,000 instances)
* RMSE of KNNWithZScore   <br />RMSE : 1.29(User based, with 55,000 instances)
* RMSE of KNNBaseline     <br />RMSE : 1.34(User based, with 55,000 instances)

It is bit difficult to show all the results in one notbook because modeling consumes all available memory, i obtained these results by commenting all others and running one algorithm at a time. 


# Reference
This notebook is greatly inspired by [this kaggle ](https://www.kaggle.com/saurav9786/recommender-system-using-amazon-reviews) notebook.